In [1]:
from graph_traffic.train_gnn import train_with_args
from graph_traffic.model_selection import get_node_ids
from random import choice

C:\Users\Bened\PycharmProjects\madrid-traffic\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ids_to_use = get_node_ids(longitud_lims=(-3.744399888121435, -3.73088591780862),
                          latitud_lims=(40.383067309465936, 40.3904))

In [3]:
len(ids_to_use)

36

## 0. Define training parameters

In [4]:
args_alternatives = {
    "diffsteps": [2, 4, 6, 8],
    "lr": [0.001, 0.005, 0.01, 0.05, 0.1, 1],
    "max_grad_norm": [1, 5, 10],
    "out_feats": [32, 64, 2*64, 4*64],
    "num_layers": [2, 4, 6, 8],
    "dir": ["both", "in", "out"],
}
data_alternatives = {
    "seq_len": [2, 4, 8, 16],
    "graph_weight_threshold": [0.1, 0.3, 0.5, 0.7]
}
alternatives_temporal = {
    "working_day": ["drop", "passthrough"],
    "hour": ["one_hot", "spline", "drop"],
}

In [5]:
args = dict(
    n_points=None,
    batch_size=64,
    diffsteps=2,
    decay_steps=2000,
    lr=0.01,
    minimum_lr=2e-6,
    epochs=10,
    max_grad_norm=5.0,
    num_workers=0,
    model="dcrnn",
    gpu=-1,
    num_heads=2, # relevant for model="gaan"
    out_feats=32,
    num_layers=4,
    dir="both",
)

data_dict = dict(
    ids_list=ids_to_use,#[3532, 3542, 3544, 3545, 3546, 3577, 3637, 3640, 3641, 4761, 4763],#[3954, 3973],#, 3976, 3977, 3978],
    seq_len=4*1,
    with_graph=True,
    from_date="2019-01-01",
    to_date="2019-12-31",
    target="ocupacion",  # 1
    interactions="drop",
    graph_weight_threshold=0.3
)

meteo_dict = dict(
    rain="drop",  # 1
    wind="drop",
    temperature="drop",  # 0
    humidity="drop",  # 1
    pressure="drop",  # 0
    radiation="drop"  # 0
)

temporal_dict = dict(
    season="drop",
    month="drop",
    day_of_month="drop",
    hour="drop",
    bank_holiday="drop",
    working_day="drop",
    school_holiday="drop",
    state_of_alarm="drop",
    minute="drop",
    year="drop",
    weekday="drop"
)

## 1 Train model

In [ ]:
for i in range(100):
    for key, values in args_alternatives.items():
        args[key] = choice(args_alternatives[key])
    for key, values in data_alternatives.items():
        data_dict[key] = choice(data_alternatives[key])
    for key, values in alternatives_temporal.items():
        temporal_dict[key] = choice(alternatives_temporal[key])
    print(args)
    print(data_dict)
    print(temporal_dict)
    args["dir"] = "both"
    train_with_args(args, data_dict, meteo_dict, temporal_dict)

{'n_points': None, 'batch_size': 64, 'diffsteps': 4, 'decay_steps': 2000, 'lr': 0.05, 'minimum_lr': 2e-06, 'epochs': 10, 'max_grad_norm': 1, 'num_workers': 0, 'model': 'dcrnn', 'gpu': -1, 'num_heads': 2, 'out_feats': 64, 'num_layers': 2, 'dir': 'in'}
{'ids_list': [5126, 9847, 5045, 3685, 5046, 5125, 5110, 5061, 5122, 5047, 5055, 5054, 5108, 3686, 5050, 5051, 5048, 10025, 3576, 5105, 5121, 5044, 5120, 5049, 5118, 3628, 3572, 5114, 3684, 5102, 3481, 5100, 5119, 5101, 5099, 5109], 'seq_len': 4, 'with_graph': True, 'from_date': '2019-01-01', 'to_date': '2019-12-31', 'target': 'ocupacion', 'interactions': 'drop', 'graph_weight_threshold': 0.5}
{'season': 'drop', 'month': 'drop', 'day_of_month': 'drop', 'hour': 'one_hot', 'bank_holiday': 'drop', 'working_day': 'drop', 'school_holiday': 'drop', 'state_of_alarm': 'drop', 'minute': 'drop', 'year': 'drop', 'weekday': 'drop'}
Shape of train_x: (5212, 4, 36, 25)
Shape of train_y: (5212, 4, 36, 25)
Shape of test_x: (1303, 4, 36, 25)
Shape of test_y

C:\Users\Bened\PycharmProjects\madrid-traffic\env\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch: 0 Train MAE: 5.666244553356636 Train MSE: 103.19825169534381 Test MAE: 6.120941298348563 Test MSE: 109.71493966246709
Epoch: 1 Train MAE: 6.189812631141849 Train MSE: 104.26759588249674 Test MAE: 6.692381472814651 Test MSE: 111.2969129162878
Epoch: 2 Train MAE: 12.012959108120057 Train MSE: 371.60380787126036 Test MAE: 12.660208384195963 Test MSE: 405.07711099122196
Epoch: 3 Train MAE: 14.341359731627673 Train MSE: 417.0484429946629 Test MAE: 15.038422993251256 Test MSE: 453.96455743536256
Epoch: 4 Train MAE: 12.650435831488632 Train MSE: 360.0932005533936 Test MAE: 13.356255349658785 Test MSE: 394.5201246559648
Epoch: 5 Train MAE: 12.521989694455774 Train MSE: 350.6154324103709 Test MAE: 13.228944369724818 Test MSE: 384.4428560792412
Epoch: 6 Train MAE: 12.745779258448904 Train MSE: 355.0617871500885 Test MAE: 13.284521965753465 Test MSE: 381.82820451478517
Epoch: 7 Train MAE: 12.216021142354826 Train MSE: 332.3410587029109 Test MAE: 12.798325266156878 Test MSE: 359.72816491993

C:\Users\Bened\PycharmProjects\madrid-traffic\env\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch: 0 Train MAE: 7.656305911929109 Train MSE: 182.7696436609411 Test MAE: 8.103092800487172 Test MSE: 196.11114845694294
Epoch: 1 Train MAE: 7.144412561904552 Train MSE: 173.61063095818682 Test MAE: 7.486831058155406 Test MSE: 183.47355105376735
Epoch: 2 Train MAE: 6.268126143965611 Train MSE: 114.00392701790237 Test MAE: 6.447848406704989 Test MSE: 121.406443623856
Epoch: 3 Train MAE: 7.269805420276731 Train MSE: 135.3742778718181 Test MAE: 7.720820470289751 Test MSE: 146.21272437985078
Epoch: 4 Train MAE: 5.159313767455345 Train MSE: 88.26770070956894 Test MAE: 5.292445616288618 Test MSE: 89.02543057352555
Epoch: 5 Train MAE: 4.999002268148023 Train MSE: 84.87727893420198 Test MAE: 5.120134657079523 Test MSE: 84.18578197756797
Epoch: 6 Train MAE: 4.913803721583167 Train MSE: 81.57141154741922 Test MAE: 5.014704834331166 Test MSE: 79.50529736612683
Epoch: 7 Train MAE: 4.858537208202273 Train MSE: 79.36642195907774 Test MAE: 4.9384072043678975 Test MSE: 77.37157346394908
Epoch: 8 Tr

C:\Users\Bened\PycharmProjects\madrid-traffic\env\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch: 0 Train MAE: 6.373066402063137 Train MSE: 130.35512157721865 Test MAE: 6.976477236974807 Test MSE: 144.14328769828933
Epoch: 1 Train MAE: 5.099200056820381 Train MSE: 85.86646750929339 Test MAE: 5.271757875170026 Test MSE: 85.82580335055596
Epoch: 2 Train MAE: 4.808894954076627 Train MSE: 77.743512125543 Test MAE: 5.045898187728155 Test MSE: 76.24026276745118
Epoch: 3 Train MAE: 4.733965367805667 Train MSE: 71.22448450315028 Test MAE: 4.81667457308088 Test MSE: 66.9219461709775
Epoch: 4 Train MAE: 4.606079741222103 Train MSE: 71.78334018590938 Test MAE: 4.740637256985619 Test MSE: 67.52455193564799
Epoch: 5 Train MAE: 4.579794238253338 Train MSE: 71.37274274093485 Test MAE: 4.806874706631615 Test MSE: 68.49917878807906
Epoch: 6 Train MAE: 4.569285270644397 Train MSE: 71.35363529653209 Test MAE: 4.796713079724993 Test MSE: 70.34590351085495
Epoch: 7 Train MAE: 4.561809208334946 Train MSE: 70.92482184449581 Test MAE: 4.7524125008356 Test MSE: 67.6881705994747
Epoch: 8 Train MAE: 4

C:\Users\Bened\PycharmProjects\madrid-traffic\env\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch: 0 Train MAE: 7.502569575642431 Train MSE: 109.51298316583421 Test MAE: 7.36939170143821 Test MSE: 105.58834613567927
Epoch: 1 Train MAE: 6.231857687927956 Train MSE: 100.04430677743181 Test MAE: 6.188370531255549 Test MSE: 94.83467174873755
Epoch: 2 Train MAE: 5.740297860877458 Train MSE: 99.41847265570253 Test MAE: 5.815776391462847 Test MSE: 96.96838695668026
